In [ ]:
#Run this cell and restart
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-629c475c-6342-a3f5-75dd-c6f45075716d)


In [ ]:
!add-apt-repository -y ppa:jonathonf//ffmpeg-4
!apt update
!apt install  -y ffmpeg

Unable to handle repository shortcut 'ppa:jonathonf//ffmpeg-4'
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [662 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-securit

In [ ]:
!pip install datasets>=2.6.1

!pip install git+https://github.com/huggingface/transformers

!pip install librosa

!pip install jiwer

!pip install evaluate>=0.30

!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-z395rzhg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-z395rzhg
  Resolved https://github.com/huggingface/transformers to commit 3f69f415adcbdaedec154ba8eac220ef3276975d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8404406 sha256=e8bf43f61122f31c29a5e3c863e4ffbe3e13d1d63072ab056f73ac5441a00d28
  Stored in directory: /tmp/pip-ephem-wheel-cache-8a_eysca/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from dataclasses import dataclass

from typing import Any, Dict, List, Union

import torch

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict


common_voice=DatasetDict()

common_voice["train"]=load_dataset("mozilla-foundation/common_voice_11_0", "pa-IN", split="train+validation", use_auth_token=True)

common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "pa-IN", split="test", use_auth_token=True)



common_voice = common_voice.remove_columns(["accent",

"age",

"client_id", "down_votes",

"gender",

"locale",

"path",

"segment",

"up_votes"])

print(common_voice)

In [ ]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer


feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Punjabi", task="transcribe")

In [ ]:
from transformers import WhisperProcessor

processor=WhisperProcessor.from_pretrained("openai/whisper-small", language="Punjabi", task="transcribe")

In [ ]:
processor

In [ ]:
print('| Check the random audio example from Common Voice dataset to see what form the data is in:')

print(f'{common_voice["train"][0]}\n')

from datasets import Audio
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

print('| Check the effect of downsampling:')

print(f'{common_voice["train"][0]}\n')

def prepare_dataset(batch):
  audio = batch ["audio"]
  batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features [0]
  batch["labels"] = tokenizer(batch["sentence"]).input_ids
  return batch


common_voice = common_voice.map(
    prepare_dataset, remove_columns=common_voice.column_names["train"],num_proc=2)# num_proc > 1 will enable multiprocessing

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
  processor: Any
  def __call__(self, features: List [Dict[str, Union [List[int], torch.Tensor]]])  -> Dict[str, torch.Tensor]:
    input_features = [{"input_features": feature ["input_features"]} for feature in features]
    batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
    label_features = [{"input_ids": feature["labels"]} for feature in features]
    labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

    labels=labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1),-100)

    if (labels[:, 0]==self.processor.tokenizer.bos_token_id).all().cpu().item():
      labels=labels[:, 1:]
    batch["labels"]=labels
    return batch

data_collator=DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
data_collator

In [ ]:
import evaluate
metric=evaluate.load("wer")

In [ ]:
metric

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
model

In [ ]:
model.config.forced_decoder_ids=None
model.config.suppress_tokens=[]

In [ ]:
from transformers import Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-pa-IN",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2, # increase by 2x for every 2x decrease in batch size
    learning_rate=5e-5,
    warmup_steps=50,
    max_steps=3000,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=15,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False, # testing
)

In [ ]:
!pip install accelerate==0.20.1


In [ ]:
from transformers import Seq2SeqTrainer

def compute_metrics(pred):
  pred_ids = pred.predictions
  label_ids=pred.label_ids
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
  wer = 100 * metric.compute(predictions=pred_str, references=label_str)
  return {"wer": wer}

trainer=Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
print("Training is started")
trainer.train()
print("training endd")

In [ ]:
# After fine-tuning, save the model
trainer.save_model("/content/whisper-small-pa-IN")

# Load the fine-tuned model
whisper_model = WhisperForConditionalGeneration.from_pretrained("/content/whisper-small-pa-IN", local_files_only=True)


In [ ]:
processor.save_pretrained("/content/whisper-small-pa-IN")


In [ ]:
#give the input audio file at file_path
import torch
import numpy as np
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Load the processor that includes the feature extractor and tokenizer
processor = WhisperProcessor.from_pretrained("/content/whisper-small-pa-IN", language="Punjabi", task="transcribe")

# Load the ASR model
whisper_model = WhisperForConditionalGeneration.from_pretrained("/content/whisper-small-pa-IN", local_files_only=True)

# Load the audio file with the correct sampling rate and force it to be mono (single-channel)
file_path = "/content/ਮੇਰਾ ਨਾਮ ਸਵਾਤੀ ਹੈ ਤੁ.m4a"
audio_array, sampling_rate = librosa.load(file_path, sr=16000, mono=True)  # Ensure mono=True for single-channel audio

# Use the processor to prepare the input features
inputs = processor(
    audio_array,
    sampling_rate=sampling_rate,
    return_tensors="pt",
    truncation=True
)

# Get the input features
input_features = inputs["input_features"]

# Pad the input features to the desired length (3000)
target_length = 3000
input_features = torch.nn.functional.pad(input_features, (0, target_length - input_features.shape[-1]))

# Print the shape of the input features (optional)
print("Input features shape:", input_features.shape)

# Use the ASR model to transcribe the audio
result = whisper_model.generate(input_features)

# Decode the generated IDs to text
transcription = processor.batch_decode(result, skip_special_tokens=True)

# Print the transcription
print("Transcription:", transcription[0])

In [ ]:
!pip install pyaudio


In [ ]:
import torch
import numpy as np
import librosa
import sounddevice as sd  # Use sounddevice for audio recording
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Load the processor that includes the feature extractor and tokenizer
processor = WhisperProcessor.from_pretrained("/content/whisper-small-pa-IN", language="Punjabi", task="transcribe")

# Load the ASR model
whisper_model = WhisperForConditionalGeneration.from_pretrained("/content/whisper-small-pa-IN", local_files_only=True)

# Set the sampling rate for recording
sampling_rate = 16000

# Record audio from the microphone
duration = 10  # You can adjust the duration based on your needs
print("Recording... Speak now!")
audio_array = sd.rec(int(sampling_rate * duration), samplerate=sampling_rate, channels=1, dtype=np.int16)
sd.wait()  # Wait for recording to complete

# Convert the audio to floating point and ensure mono
audio_array = audio_array.flatten().astype(np.float32)

# Use the processor to prepare the input features
inputs = processor(
    audio_array,
    sampling_rate=sampling_rate,
    return_tensors="pt",
    truncation=True
)

# Get the input features
input_features = inputs["input_features"]

# Pad the input features to the desired length (3000)
target_length = 3000
input_features = torch.nn.functional.pad(input_features, (0, target_length - input_features.shape[-1]))

# Print the shape of the input features (optional)
print("Input features shape:", input_features.shape)

# Use the ASR model to transcribe the audio
result = whisper_model.generate(input_features)

# Decode the generated IDs to text
transcription = processor.batch_decode(result, skip_special_tokens=True)

# Print the transcription
print("Transcription:", transcription[0])


In [ ]:
from pydub import AudioSegment

# Load stereo audio
audio = AudioSegment.from_file("/content/WhatsApp Ptt 2023-12-15 at 10.23.06 PM.wav", format="wav")

# Convert to mono
mono_audio = audio.set_channels(1)

# Export as mono
mono_audio.export("mono_file.wav", format="wav")
